# Finalize StyleGAN Dataset

by @duskvirkus

A simple notebook for collecting images from many directories into one directory and verifing that they all match the desired dimensions.

## Configurations
Change this section for your dataset. Then run other cells you shouldn't need to change anything else unless you want to.

In [ ]:
directories = [ # Add all directories you want to pull images from.
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-3-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-4-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-5-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-6-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-7-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-8-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-9-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-10-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-11-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-12-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-17-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-18-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-19-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-20-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-21-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-22-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-24-reviewed',
    '/home/duskvirkus/dataset-creation/grimes-reviewed/faces-grimes-vid-25-reviewed',
]

In [ ]:
dataset_name = 'grimes'
dataset_version = 1
destination_dir = '/home/duskvirkus/datasets/grimes-v1'
desired_img_size = [1024, 1024] # [x, y] dimensions

## Download Dependencies and Imports

In [ ]:
import os

root_dir = os.getcwd()

vendor_path = os.path.join(root_dir, 'vendor')
dst_path = os.path.join(vendor_path, 'dataset-tools');
nbt_path = os.path.join(vendor_path, 'notebook-utils');

if not os.path.exists(dst_path):
    if not os.path.exists(vendor_path):
        os.makedirs(vendor_path)
    %cd {vendor_path}
    !git clone https://github.com/duskvirkus/dataset-tools.git
    %cd {root_dir}
    
if not os.path.exists(nbt_path):
    %cd {vendor_path}
    !git clone https://github.com/duskvirkus/notebook-utils
    %cd {root_dir}
    
%cd {dst_path}
!pip install -r requirements.txt
%cd {root_dir}

import cv2
import sys
sys.path.insert(1, nbt_path)
from user_input import yes_or_no

## Collate into one folder

In [ ]:
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)
    
def get_file_number(current, digits):
    ret = str(current)
    while (len(ret) < digits):
        ret = '0' + ret
    return ret
    
save_count = 0
for directory in directories:
    all_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            extension = file.split('.')[-1]
            all_files.append([os.path.join(root,file), extension])
    for i in range(len(all_files)):
        file = all_files[i][0]
        ext = all_files[i][1]
        save_path = os.path.join(destination_dir, dataset_name + '-v' + str(dataset_version) + '-' + get_file_number(save_count, 5) + '.' + ext)
        !cp {file} {save_path}
        print(save_path + ' saved!')
        save_count += 1

## Verify Image Size

In [ ]:
for root, dirs, files in os.walk(destination_dir):
    for file in files:
        path = os.path.join(root,file)
        img = cv2.imread(path)
        if img.shape[1] != desired_img_size[0] or img.shape[0] != desired_img_size[1]:
            print('warning - file: ' + path + ' does not match desired size!')
            if yes_or_no('Would you like to remove it from the dataset?'):
                !rm {path}

## Zip for Upload

In [ ]:
%cd {destination_dir}
zip_name = '../' + dataset_name + '-v' + str(dataset_version) + '.zip'
!zip {zip_name} *.*
%cd {root_dir}